# An implicit linear shallow water model

The non-linear rotating shallow water system is a fundamental equation set in geophysical fluid dynamics. A complete non-linear method will often require a sequence of solutions to a linearized model (Newton-type methods). Therefore, in this notebook we will look at solvers for the linear shallow water model on the sphere:

$$
\frac{\partial \mathbf{u}}{\partial t} + f\mathbf{u}^{\perp} + g\nabla D = 0, \\
\frac{\partial D}{\partial t} + H\nabla \cdot \mathbf{u} = 0.
$$

This defines a coupled set of equations for a velocity $\mathbf{u}$ and depth $D$. Here, $f$ is the Coriolis parameter, $g$ is the acceleration due to gravity, and $H$ is a mean layer depth. Our equations are defined on a two-dimensional sphere embedded in $\mathbb{R}^3$, $\Omega$.

## Spatial discretization

Now we discretize in space. First, we mulitply the first equation by a test function $\mathbf{w} \in V \subset H(\text{div}; \Omega)$, and the second equation by a test function $\phi \in U \subset L^2(\Omega)$. Integrating both equations by parts, we obtain the following semi-discrete problem: find $\mathbf{u}(x, t), D(x, t) \in V \times U$ such that:

$$
\int_\Omega \mathbf{w}\cdot\left(\frac{\partial\mathbf{u}}{\partial t} + f\mathbf{u}^{\perp}\right)\, \mathrm{d}x - g\int_\Omega D\nabla\cdot\mathbf{w}\, \mathrm{d}x = 0, \quad \forall \mathbf{w} \in V,\\
\int_\Omega \phi \frac{\partial D}{\partial t}\,\mathrm{d}x + H\int_\Omega \phi \nabla\cdot\mathbf{u}\,\mathrm{d}x = 0, \quad \forall \phi \in U.
$$

## Time discretization (implicit midpoint rule)
We now need to discretize in time. Using the implicit midpoint method, solving the time-dependent problem requires successive solutions to the spatial discretization at time $t_{n+1} = t_{n} + \Delta t$ for $u^{n+1}, D^{n+1} \in V \times U$ satifying

$$
\int_\Omega \mathbf{w}\cdot\mathbf{u}^{n+1}\,\mathrm{d}x + \frac{\Delta t}{2}\int_\Omega \mathbf{w}\cdot f\mathbf{u}^{n+1}\,\mathrm{d}x - \frac{g\Delta t}{2}\int_\Omega D^{n+1}\nabla\cdot\mathbf{w}\,\mathrm{d}x = -R^n\lbrack \mathbf{w} \rbrack, \quad \forall \mathbf{w} \in V, \\
\int_\Omega \phi D^{n+1}\,\mathrm{d}x + \frac{H\Delta t}{2}\int_\Omega \phi\nabla\cdot\mathbf{u}^{n+1}\,\mathrm{d}x = -R^n\lbrack \phi \rbrack, \quad \forall \phi \in V,
$$

where the residual co-vectors are defined as:

$$
R^n\lbrack \mathbf{w} \rbrack = \int_\Omega \mathbf{w}\cdot\mathbf{u}^{n}\,\mathrm{d}x - \frac{\Delta t}{2}\int_\Omega \mathbf{w}\cdot f\mathbf{u}^{n}\,\mathrm{d}x + \frac{g\Delta t}{2}\int_\Omega D^{n}\nabla\cdot\mathbf{w}\,\mathrm{d}x \\
R^n\lbrack \phi \rbrack = \int_\Omega \phi D^{n}\,\mathrm{d}x - \frac{H\Delta t}{2}\int_\Omega \phi\nabla\cdot\mathbf{u}^{n}\,\mathrm{d}x.
$$

For each time-step, we require the numerical solution to the saddle-point system:

$$
\mathbf{A}\mathbf{x} \equiv \begin{bmatrix} A & -\frac{g\Delta t}{2}B^T \\ \frac{H\Delta t}{2} B & C \end{bmatrix} \begin{Bmatrix} U^{n+1} \\ D^{n+1}\end{Bmatrix} = \begin{Bmatrix} -R^n\lbrack \mathbf{w} \rbrack \\ -R^n\lbrack \phi \rbrack \end{Bmatrix} \equiv \mathbf{b}
$$

for the coefficent vectors $U^{n+1}$ and $D^{n+1}$. One standard approach for such systems revolves around preconditioning the operator $\mathbf{A}$ by an approximate inverse of the its Schur-complement factorization. In this notebook, we will highlight a techniqued known as "hybridization," whereby a different discretization (approximating the same system) replaces the original $H(\text{div})\times L^2$ mixed method by a "hybrid" version. This new discretization permits the use of element-wise static condensation to exactly eliminate the prognostic unknowns for a reduced problem on mesh interfaces.

## Implementation

To solve the problem in a concrete setting, we need two things.  A domain, and initial conditions for $\mathbf{u}$ and $D$. Our domain will be a spherical mesh consisting of simplices and a cubic coordinate field. This way, we can have a suitable approximation of the curvature.

We begin by importing Firedrake and defining some relevant constants.

In [ ]:
%matplotlib inline
from firedrake import *

R0 = 6371220.0
R = Constant(R0)              # Radius of the earth [m]
H = Constant(5960.0)          # Mean depth [m]
day = 24. * 60. * 60.         # Seconds in a day [s]
Omega_f = Constant(7.292E-5)  # Angular rotation rate [rads]
g = Constant(9.80616)         # Acceleration due to gravity [m/s^2]
mesh_degree = 3               # Cubic coordinate field
refinements = 3               # Number of icosahedron refinements
mesh = IcosahedralSphereMesh(R0, refinements,
                             degree=mesh_degree)

Now we initialize and orient the coordinates on the sphere:

In [ ]:
x = SpatialCoordinate(mesh)
global_normal = as_vector([x[0], x[1], x[2]])
mesh.init_cell_orientations(global_normal)

If one desires, you can optionally plot the generated mesh:

In [ ]:
plot(mesh);

We now define the mixed method method by choosing our discrete function spaces. We use a standard BDM mixed method, taking $V = BDM_2$ and $U = DG_1$:

In [ ]:
V = FunctionSpace(mesh, "BDM", 2)
U = FunctionSpace(mesh, "DG", 1)
W = V * U

Now we add the Coriolis term as a function in a continuous finite element space:

In [ ]:
f_expr = 2 * Omega_f * x[2] / R
Vf = FunctionSpace(mesh, "CG", mesh_degree)
f = Function(Vf).interpolate(f_expr)    # Coriolis frequency [1/s]

For our initial conditions, we choose profiles for $\mathbf{u}$ and $D$ such that they are in a state of geostrophic balance:

$$
\mathbf{u}(\mathbf{x}, 0) = \frac{u_{\text{max}}}{R_{\odot}}\left( -y, x, 0\right), \\
D(\mathbf{x}, 0) = H - \left(R_\odot\Omega_f u_{\text{max}} + \frac{u_{\text{max}}^2}{2}\right)\frac{z^2}{gR^2_\odot}
$$

where $u_{\text{max}} = 2\pi R_\odot/(12 \text{days}) \approx 38.6 \text{m}\text{s}^{-1}$. Since there is no forcing or topography, the solution should remain steady throughout the entire simulation. In Firedrake, we simply define the expressions in UFL and project/interpolate into the relevant finite element spaces:

In [ ]:
u_0 = 2*pi*R0/(12*day)
u_max = Constant(u_0)                 # Maximum amplitude of the zonal wind [m/s]
u_expr = as_vector([-u_max*x[1]/R, u_max*x[0]/R, 0.0])
D_expr = H - ((R * Omega_f * u_max + u_max*u_max/2.0)*(x[2]*x[2]/(R*R)))/g
u0 = Function(V).project(u_expr, solver_parameters={'ksp_type': 'preonly',
                                                    'pc_type': 'lu',
                                                    'pc_factor_mat_solver_type': 'mumps'})
D0 = Function(U).interpolate(D_expr)

We also need solution functions for $\mathbf{u}^{n+1}$ and $\mathbf{u}^n$ (as well as $D^{n+1}$ and $D^n$). Additionally, we assign the fields with their initial values.

In [ ]:
wn = Function(W, name="w^n")          # Fields at time-step n
un, Dn = wn.split()                   # Split mixed function for individual fields
un.assign(u0)
Dn.assign(D0)
wn1 = Function(W, name="w^{n+1}")     # Fields at time-tep n+1
un1, Dn1 = wn1.split()

Now we set the maximum simulation time, time-step size, time-stepping coefficients, and the relevant test/trial functions:

In [ ]:
tmax = day
Dt = 4000.0
dt = Constant(Dt)                     # Time-step size [s]
alpha = Constant(0.5)                 # Midpoint method
u, D = TrialFunctions(W)
w, phi = TestFunctions(W)

Finally, we define the "perp" operator:

In [ ]:
outward_normal = CellNormal(mesh)
perp = lambda u: cross(outward_normal, u)

Now that we have initalized our fields defined relevant parameters, we define our finite element problem in residual form:

In [ ]:
uD_eqn = (inner(w, u) + alpha*dt*inner(w, f*perp(u))
          - alpha*dt*g*div(w)*D
          - inner(w, un)
          + alpha*dt*inner(w, f*perp(un))
          - alpha*dt*g*div(w)*Dn
          + phi*D + alpha*dt*H*phi*div(u)
          - phi*Dn + alpha*dt*H*phi*div(un))*dx

And our problem/solver context with a Schur-complement preconditioner:

In [ ]:
uD_problem = LinearVariationalProblem(lhs(uD_eqn), rhs(uD_eqn), wn1)
solver_parameters = {'ksp_type': 'gmres',
                     'ksp_rtol': 1.0e-8,
                     'pc_type': 'fieldsplit',
                     'pc_fieldsplit_type': 'schur',
                     'pc_fieldsplit_schur_fact_type': 'full',
                     'pc_fieldsplit_schur_precondition': 'selfp',
                     'fieldsplit_0': {'ksp_type': 'preonly',
                                      'pc_type': 'bjacobi',
                                      'sub_pc_type': 'ilu'},
                     'fieldsplit_1': {'ksp_type': 'preonly',
                                      'pc_type': 'gamg',
                                      'mg_levels': {'ksp_type': 'chebyshev',
                                                    'ksp_max_it': 2,
                                                    'pc_type': 'bjacobi',
                                                    'sub_pc_type': 'ilu'}}}
uD_solver = LinearVariationalSolver(uD_problem,
                                    solver_parameters=solver_parameters)

Now we simply execute the time loop:

In [ ]:
t = 0.0
while t < tmax:        # Start time loop
    t += Dt
    uD_solver.solve()  # Solve for updated fields
    un.assign(un1)     # Update fields for next time-step
    Dn.assign(Dn1)

Since the flow is steady, the relative error in the fields compared with their initial values should be small (relative to discretization error). We can do this using Firedrake's `errornorm` and `norm` functions:

In [ ]:
Uerr = errornorm(u_expr, un1, norm_type="L2")/norm(u_expr, norm_type="L2")
Derr = errornorm(D_expr, Dn1, norm_type="L2")/norm(D_expr, norm_type="L2")
print("Error in the velocity: %s" % Uerr)
print("Error in the depth: %s" % Derr)

### Solver convergence

Although the code we wrote above works fine, let's take a closer look at the solver by turning on PETSc's KSP monitor. Resetting back to the initial state, we now turn on the monitor and run for a single time-step:

In [ ]:
# NBVAL_SKIP
solver_parameters['ksp_monitor_true_residual'] = True
from firedrake.solving_utils import KSPReasons
un.assign(u0)
Dn.assign(D0)
wn1.assign(0.0)
uD_solver = LinearVariationalSolver(uD_problem,
                                    solver_parameters=solver_parameters)
uD_solver.solve()
print("gmres iterations = {}, converged reason = {}".format(
       uD_solver.snes.ksp.getIterationNumber(), 
       KSPReasons[uD_solver.snes.ksp.getConvergedReason()]))